# Merging bases commentaires

In [1]:
import pandas, os

On récupère les fichiers csv de nos différents dossiers pour les fusionner en une seule base.

In [2]:
df_general = pandas.DataFrame()

TA = "Data/Bases TripAdvisor/"
print ("Nombre de fichiers TripAdvisor:", len(os.listdir(TA)))
for filename in os.listdir(TA):
    df = pandas.read_csv(TA + filename, sep = ";", encoding = 'utf_8', index_col = 0)
    df['Origine'] = "TripAdvisor"
    df_general = pandas.concat( (df_general, df) )
    
Yelp = "Data/Bases Yelp/"
print ("Nombre de fichiers Yelp:", len(os.listdir(Yelp)))
for filename in os.listdir(Yelp) :
    df = pandas.read_csv(Yelp + filename, encoding = "utf-8")
    df['Resto'] = df['\ufeffResto'] # règle un problème avant le début du document et l'encoding en utf-8
    del df['\ufeffResto']
    df['Origine'] = 'Yelp'
    df_general = pandas.concat( (df_general, df) )    
    
print(df_general.shape)
df_general.head()

Nombre de fichiers TripAdvisor: 125
Nombre de fichiers Yelp: 1
(85897, 8)


,Adresse,Code Postal,Commentaire,Date du commentaire,Note,Origine,Resto,Ville
0,63 AV MOZART,75016,\nUn accueil hyper chaleureux! Les gérants son...,30/04/2016,5,TripAdvisor,MACIS CAFE,Paris
1,63 AV MOZART,75016,"\nNous cherchions à déjeuner, seul bémol, l'he...",21/04/2016,4,TripAdvisor,MACIS CAFE,Paris
2,63 AV MOZART,75016,\nDes plats réalisés à partir de produits frai...,10/02/2016,3,TripAdvisor,MACIS CAFE,Paris
0,90 Rue des Orteaux,75020,\nRestaurant Sushi plutot correct dans l ensem...,27/02/2016,3,TripAdvisor,Sushi Tomi,Paris
1,90 Rue des Orteaux,75020,\nDéçue de ma dernière visite car impossible d...,17/01/2016,3,TripAdvisor,Sushi Tomi,Paris


In [3]:
print(df_general[df_general.Origine == "Yelp"].shape)
print(df_general[df_general.Origine == "TripAdvisor"].shape)

(323, 8)
(85574, 8)


On remet l'index à jour.

In [4]:
df_general.reset_index(drop=True, inplace=True)

On donne un identifiant unique à chaque commentaire.

In [5]:
df_general["Id"] = df_general.index

On corrige le format des commentaires et du nom des restaurants.

In [6]:
for i in range(df_general.shape[0]):
    df_general.at[i, "Commentaire"] = df_general.at[i, "Commentaire"].strip().replace("\r", "").replace("\n", " ").replace("Ã©", "é").replace("Ã", "à")
    df_general.at[i, "Resto"] = df_general.at[i, "Resto"].lower()

On harmonise la date des commentaires.

In [7]:
for i in range(df_general.shape[0]):
    x = df_general.at[i, "Date du commentaire"].split("/")
    df_general.at[i, "Date"] = int(x[2] + x[1] + x[0])

In [8]:
df_general.head()

,Adresse,Code Postal,Commentaire,Date du commentaire,Note,Origine,Resto,Ville,Id,Date
0,63 AV MOZART,75016,Un accueil hyper chaleureux! Les gérants sont ...,30/04/2016,5,TripAdvisor,macis cafe,Paris,0,20160430
1,63 AV MOZART,75016,"Nous cherchions à déjeuner, seul bémol, l'heur...",21/04/2016,4,TripAdvisor,macis cafe,Paris,1,20160421
2,63 AV MOZART,75016,Des plats réalisés à partir de produits frais ...,10/02/2016,3,TripAdvisor,macis cafe,Paris,2,20160210
3,90 Rue des Orteaux,75020,Restaurant Sushi plutot correct dans l ensembl...,27/02/2016,3,TripAdvisor,sushi tomi,Paris,3,20160227
4,90 Rue des Orteaux,75020,Déçue de ma dernière visite car impossible de ...,17/01/2016,3,TripAdvisor,sushi tomi,Paris,4,20160117


On enregistre la base.

In [9]:
df_general.to_csv("base_DFG.csv", index = False, encoding = 'utf-8')